In [28]:
import numpy as np

n = 5
p = 8

A = np.random.randint(2, 5, (n, n))


X = np.full(n, p)

B = A @ X

print(B)
print(X)
print(A)
np.save("matrix_A.npy", A)
np.save("vector_B.npy", B)


[136 112 120 128 144]
[8 8 8 8 8]
[[3 4 3 4 3]
 [2 3 2 3 4]
 [2 3 3 4 3]
 [4 2 4 3 3]
 [3 4 4 3 4]]


In [29]:
def LU_find(L, U, A, n):
    for k in range(n):
        for i in range(k, n):
            sum_ = sum(L[i][j] * U[j][k] for j in range(k))
            L[i][k] = A[i][k] - sum_

        for i in range(k + 1, n):
            sum_ = sum(L[k][j] * U[j][i] for j in range(k))
            U[k][i] = (A[k][i] - sum_) / L[k][k]


def LU_solve(L, U, B, n):
    Z = np.zeros(n)
    X = np.zeros(n)

    for k in range(n):
        Z[k] = (B[k] - sum(L[k][j] * Z[j] for j in range(k))) / L[k][k]

    for k in range(n - 1, -1, -1):
        X[k] = Z[k] - sum(U[k][j] * X[j] for j in range(k + 1, n))

    return X


A = np.load("matrix_A.npy")
B = np.load("vector_B.npy")

L = np.zeros((n, n))
U = np.identity(n)
np.fill_diagonal(L, 1)

LU_find(L, U, A, n)

X_computed = LU_solve(L, U, B, n)
print(X_computed)
print("Matrix L:")
print(L)
print("\nMatrix U:")
print(U)

[8. 8. 8. 8. 8.]
Matrix L:
[[ 3.          0.          0.          0.          0.        ]
 [ 2.          0.33333333  0.          0.          0.        ]
 [ 2.          0.33333333  1.          0.          0.        ]
 [ 4.         -3.33333333  0.          1.          0.        ]
 [ 3.          0.          1.         -2.         40.        ]]

Matrix U:
[[ 1.          1.33333333  1.          1.33333333  1.        ]
 [ 0.          1.          0.          1.          6.        ]
 [ 0.          0.          1.          1.         -1.        ]
 [ 0.          0.          0.          1.         19.        ]
 [ 0.          0.          0.          0.          1.        ]]


In [30]:
B_recomputed = A @ X_computed
eps = np.max(np.abs(B_recomputed - B))
print(f"eps = {eps:.6e}")

eps = 2.842171e-14


In [31]:
# === Ітераційне уточнення ===
eps_target = 1e-14
X_cur = X_computed.copy()
iteration = 0

while True:
    R = B - A @ X_cur  # Вектор нев'язки
    if np.max(np.abs(R)) < eps_target:
        break

    # Розв'язуємо A * ΔX = R через LU
    Z_corr = np.zeros(n)
    DX = np.zeros(n)

    # Прямий хід
    for k in range(n):
        Z_corr[k] = (R[k] - sum(L[k][j] * Z_corr[j] for j in range(k))) / L[k][k]

    # Зворотний хід
    for k in range(n - 1, -1, -1):
        DX[k] = Z_corr[k] - sum(U[k][j] * DX[j] for j in range(k + 1, n))

    # Уточнюємо X
    X_cur += DX
    iteration += 1

# Вивід результатів
print("\n--- Ітераційне уточнення ---")
print("Уточнений розв'язок:")
print(X_cur)
print(f"Кількість ітерацій: {iteration}")
print(f"Макс. нев'язка після уточнення: {np.max(np.abs(B - A @ X_cur)):.2e}")



--- Ітераційне уточнення ---
Уточнений розв'язок:
[8. 8. 8. 8. 8.]
Кількість ітерацій: 2
Макс. нев'язка після уточнення: 0.00e+00
